In [4]:
from google.colab import files
upload=files.upload()

Saving train_2kmZucJ.csv to train_2kmZucJ (2).csv
Saving test_oJQbWVk.csv to test_oJQbWVk (1).csv


datasets:-https://datahack.analyticsvidhya.com/contest/linguipedia-codefest-natural-language-processing-1/


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
train=pd.read_csv("train_2kmZucJ.csv")
test=pd.read_csv("test_oJQbWVk.csv")

In [7]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [8]:
test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [9]:
train.shape,test.shape,train['label'].shape

((4439, 3), (1953, 2), (4439,))

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4439 entries, 0 to 4438
Data columns (total 3 columns):
id       4439 non-null int64
label    4439 non-null int64
tweet    4439 non-null object
dtypes: int64(2), object(1)
memory usage: 104.2+ KB


In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1953 entries, 0 to 1952
Data columns (total 2 columns):
id       1953 non-null int64
tweet    1953 non-null object
dtypes: int64(1), object(1)
memory usage: 30.6+ KB


In [12]:
train['label'].value_counts()

0    3305
1    1134
Name: label, dtype: int64

In [13]:
train['label'].unique()

array([0, 1])

In [14]:
import re
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk import chunk
import nltk
nltk.download('stopwords')
stop=stopwords.words('english')
nltk.download("wordnet")
lem=WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [0]:
train_new_data=[]
for data in train['tweet']:
  data=data.lower()
  data=re.sub(r'[^a-z0-9]',' ',data)
  data=re.sub(r'\s+',' ',data)
  data=[lem.lemmatize(word) for word in data.split() if word not in stop]
  data=' '.join(data)
  train_new_data.append(data)

In [0]:
test_new_data=[]
for data in test['tweet']:
  data=data.lower()
  data=re.sub(r'[^a-z0-9]',' ',data)
  data=re.sub(r'\s+',' ',data)
  data=[lem.lemmatize(word) for word in data.split() if word not in stop]
  data=' '.join(data)
  test_new_data.append(data)
   

In [0]:
tfidf=TfidfVectorizer()
cv=CountVectorizer()

In [0]:
x_tf_train=tfidf.fit_transform(train_new_data).toarray()
x_cv_train=cv.fit_transform(train_new_data).toarray()
x_tf_test=tfidf.fit_transform(train_new_data).toarray()
x_cv_test=cv.fit_transform(train_new_data).toarray()

In [20]:
x_cv_train.shape,x_tf_train.shape

((4439, 14473), (4439, 14473))

In [21]:
x_cv_test.shape,x_tf_test.shape

((4439, 14473), (4439, 14473))

In [0]:
y=train['label']

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [24]:
dec=DecisionTreeClassifier()
dec.fit(x_tf_train,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:
predict=dec.predict(x_cv_train)

In [26]:
print(classification_report(predict,y))
print(confusion_matrix(predict,y))
print(accuracy_score(predict,y))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93      3614
           1       0.64      0.88      0.74       825

    accuracy                           0.89      4439
   macro avg       0.81      0.89      0.84      4439
weighted avg       0.91      0.89      0.89      4439

[[3209  405]
 [  96  729]]
0.8871367425095742


In [27]:
mul=MultinomialNB()
mul.fit(x_tf_train,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
predict1=mul.predict(x_tf_train)

In [29]:
print(classification_report(predict1,y))
print(confusion_matrix(predict1,y))
print(accuracy_score(predict1,y))

              precision    recall  f1-score   support

           0       0.99      0.90      0.94      3616
           1       0.69      0.96      0.80       823

    accuracy                           0.91      4439
   macro avg       0.84      0.93      0.87      4439
weighted avg       0.93      0.91      0.92      4439

[[3269  347]
 [  36  787]]
0.9137193061500338


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [31]:
log=LogisticRegression()
log.fit(x_tf_train,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
predict2=log.predict(x_tf_train)

In [33]:
print(classification_report(y,predict2))
print(confusion_matrix(y,predict2))
print(accuracy_score(y,predict2))

              precision    recall  f1-score   support

           0       0.94      0.97      0.96      3305
           1       0.91      0.81      0.86      1134

    accuracy                           0.93      4439
   macro avg       0.93      0.89      0.91      4439
weighted avg       0.93      0.93      0.93      4439

[[3219   86]
 [ 213  921]]
0.9326424870466321
